In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
import torch
import pandas as pd

In [ ]:
data_en = pd.read_json('/content/drive/MyDrive/LREC-Coling/data/ML-ESG-3_Trainset_English.json')

In [ ]:
len(data_en)

545

In [ ]:
data_en.head()

,URL,news_title,news_content,impact_level,impact_length
0,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,ESG-focused financial technology company Arabe...,low,2 to 5 years
1,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,The company also announced the appointment of ...,low,2 to 5 years
2,https://www.esgtoday.com/arabesque-ai-appoints...,Arabesque AI Appoints Carolina Minio Paluello ...,Wong said: \n“Personalised portfolios demand ...,medium,2 to 5 years
3,https://www.esgtoday.com/ukraine-war-inflation...,"Ukraine War, Inflation Reduction Act Driving F...",One of the key themes of the report is the imp...,high,More than 5 years
4,https://www.esgtoday.com/eu-regulators-welcome...,"EU Regulators Welcome, Critique New European S...",Europe’s three primary financial regulatory ag...,medium,Less than 2 years


In [ ]:
data_en['impact_level'].value_counts()

medium    243
high      196
low       106
Name: impact_level, dtype: int64

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=20,
    num_beam_groups=5,
    num_return_sequences=20,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [ ]:
from tqdm.notebook import tqdm
def get_paraphrase(data, p_length = 20):
  columns = data.columns
  for i in tqdm(range(len(data))):
    temp = {}
    for col in columns:
      if col != 'news_content':
        temp[col] = [data.iloc[i][col]] * p_length
      else:
        temp[col] = paraphrase(data.iloc[i][col])
    #display(temp['news_content'])
    temp_df = pd.DataFrame(temp)
    if i == 0:
      final_df = temp_df
    else:
      final_df = pd.concat([final_df, temp_df], axis = 0)

  return final_df






In [ ]:
pphased_df = get_paraphrase(data_en)

In [ ]:
pphased_df.to_csv('/content/drive/MyDrive/LREC-Coling/data/English.csv', index = False)